In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys

sys.path.append("../")

import torch
import transformers
import baukit
from tqdm.auto import tqdm
import json
import os
from src import functional
import src.tokens as tokenization_utils
import numpy as np
import logging
from src import models

from src.utils import logging_utils
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

torch.__version__, transformers.__version__, torch.version.cuda

('2.1.2+cu121', '4.39.0.dev0', '12.1')

In [4]:
from src.models import ModelandTokenizer

# MODEL_PATH = "state-spaces/mamba-2.8b-slimpj" # state-spaces/mamba-2.8b
MODEL_PATH = "EleutherAI/pythia-2.8b-deduped"

mt = ModelandTokenizer(
    model_path=MODEL_PATH, 
    torch_dtype=torch.float32
)

2024-03-19 16:24:19 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2024-03-19 16:24:20 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /EleutherAI/pythia-2.8b-deduped/resolve/main/config.json HTTP/1.1" 200 0
2024-03-19 16:24:21 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /EleutherAI/pythia-2.8b-deduped/resolve/main/generation_config.json HTTP/1.1" 404 0
2024-03-19 16:24:22 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /EleutherAI/pythia-2.8b-deduped/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2024-03-19 16:24:22 src.models INFO     loaded model <EleutherAI/pythia-2.8b-deduped> | size: 10724.583 MB | dtype: torch.float32 | device: cuda


In [5]:
#####################################################
subject = "The Space Needle"
prompt_template = "{} is located in the city of"
# prompt_template = tokenization_utils.maybe_prefix_eos(
#     mt.tokenizer, prompt_template
# )
#####################################################

prompt = prompt_template.format(subject)
prompt

'The Space Needle is located in the city of'

In [6]:
from src.functional import predict_next_token

predict_next_token(
    mt,
    prompt=prompt,
    k=5,
)

[[PredictedToken(token=' Seattle', prob=0.9797233939170837),
  PredictedToken(token=' Portland', prob=0.0019473006250336766),
  PredictedToken(token=' Vancouver', prob=0.0017969132168218493),
  PredictedToken(token=' Iss', prob=0.0013203793205320835),
  PredictedToken(token='Se', prob=0.0013112127780914307)]]

In [7]:
# from src.data.dataclasses import MultiCounterFactDataset

# dataset = MultiCounterFactDataset("../data")

request = {
    "prompt": prompt_template,
    "subject": subject,
    "target_new": {"str": "Paris"},
}

generation_prompts = [
    f"{subject} is located in the city of",
    f"{subject}, which is in the city of",
    f"Which city is the {subject} in? It is in",
    f"{subject} is made of",
    f"{subject} is in"
]

In [8]:
mt.model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 2560)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
          (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=True)


In [9]:
from src.rome.compute_v import compute_v, get_module_input_output_at_word

context_templates=[
    '{}', 
    'The first step to a new life is to. {}', 
    'Therefore, the best way to prevent this from. {}', 
    'Because the first time I saw the trailer. {}', 
    "I'm not sure if this is the. {}", 
    'You are here: Home / Archives for . {}', 
]
words= [subject] * len(context_templates)

l_input, l_output = get_module_input_output_at_word(
    mt, 
    layer = 15,
    context_template = request["prompt"],
    word = request["subject"],
    module_template=mt.layer_name_format + ".mlp.dense_4h_to_h",
    fact_token_strategy="subject_last"
)

2024-03-19 16:24:23 numexpr.utils INFO     Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-03-19 16:24:23 numexpr.utils INFO     NumExpr defaulting to 8 threads.


2024-03-19 16:24:23 datasets INFO     PyTorch version 2.1.2 available.
2024-03-19 16:24:23 matplotlib DEBUG    matplotlib data path: /home/local_arnab/miniconda3/envs/relations/lib/python3.10/site-packages/matplotlib/mpl-data
2024-03-19 16:24:23 matplotlib DEBUG    CONFIGDIR=/home/local_arnab/.config/matplotlib
2024-03-19 16:24:23 matplotlib DEBUG    interactive is False
2024-03-19 16:24:23 matplotlib DEBUG    platform is linux
2024-03-19 16:24:23 src.rome.repr_tools DEBUG    ==> [([3], 'le')]


In [10]:
# from src.rome_utils import nethook

# tokenized = mt.tokenizer(prompt, return_tensors="pt", padding=True, return_offsets_mapping=True).to(mt.device)
# offsets = tokenized.pop("offset_mapping")

# [(idx, mt.tokenizer.decode(t)) for idx, t in enumerate(tokenized.input_ids[0])]

In [11]:
# with nethook.Trace(
#     module = mt.model,
#     layer = mt.layer_name_format.format(15) + ".mixer",
#     retain_output = True,
#     retain_input = True,
# ) as tr:
#     output = mt(**tokenized)

In [12]:
request["prompt"]

'{} is located in the city of'

In [13]:
mt.mlp_module_name_format

'gpt_neox.layers.{}.mlp'

In [14]:
from src.rome.rome_hparams import ROMEHyperParams

hparams = ROMEHyperParams(
    layers = [15],
    fact_token="subject_last",
    v_num_grad_steps=20,
    v_lr=5e-1,
    v_loss_layer=models.determine_layers(mt)[-1],
    v_weight_decay=0.5,
    clamp_norm_factor=3,
    kl_factor=0.0625,
    mom2_adjustment=True,
    context_template_length_params=[[5, 10], [10, 10]],

    rewrite_module_tmp=mt.layer_name_format + ".mlp.dense_4h_to_h",
    layer_module_tmp=mt.layer_name_format,
    mlp_module_tmp=mt.mlp_module_name_format,
    attn_module_tmp=mt.attn_module_name_format,
    ln_f_module=models.determine_final_layer_norm_path(mt),
    lm_head_module=models.determine_lm_head_path(mt),
    
    mom2_dataset="wikipedia",
    mom2_n_samples=1000,
    mom2_dtype="float32",
)

In [15]:
import json
print(json.dumps(hparams.__dict__, indent=2))
# hparams.__dict__

{
  "layers": [
    15
  ],
  "fact_token": "subject_last",
  "v_num_grad_steps": 20,
  "v_lr": 0.5,
  "v_loss_layer": 31,
  "v_weight_decay": 0.5,
  "clamp_norm_factor": 3,
  "kl_factor": 0.0625,
  "mom2_adjustment": true,
  "context_template_length_params": [
    [
      5,
      10
    ],
    [
      10,
      10
    ]
  ],
  "rewrite_module_tmp": "gpt_neox.layers.{}.mlp.dense_4h_to_h",
  "layer_module_tmp": "gpt_neox.layers.{}",
  "mlp_module_tmp": "gpt_neox.layers.{}.mlp",
  "attn_module_tmp": "gpt_neox.layers.{}.attention",
  "ln_f_module": "gpt_neox.final_layer_norm",
  "lm_head_module": "embed_out",
  "mom2_dataset": "wikipedia",
  "mom2_n_samples": 1000,
  "mom2_dtype": "float32",
  "mamba_block_non_ssm": false,
  "mamba_block_ssm": false
}


In [16]:
from src.rome.rome_main import get_context_templates

get_context_templates(
    mt = mt,
    length_params=[[5, 10], [10, 10]]
)

Cached context templates ['{}', 'Q: . {}', 'Q: . {}', 'Q: . {}', 'The present invention relates. {}', 'Q: . {}', 'Q: . {}', 'A new report from. {}', 'Q: . {}', ' How to. {}', 'Q: . {}', 'Q: How can you make a. {}', ' A little bit about me...\n. {}', 'The present invention relates to a process for preparing. {}', 'Q: How to get the value. {}', 'Q: Can you get a list. {}', 'The present invention relates generally to a semiconductor device. {}', '1. Introduction #sec1\n===============. {}', 'The present invention relates to a process for producing. {}', '1. Field of the Invention\nThe present. {}', 'Q: How can I make a. {}']


['{}',
 'Q: . {}',
 'Q: . {}',
 'Q: . {}',
 'The present invention relates. {}',
 'Q: . {}',
 'Q: . {}',
 'A new report from. {}',
 'Q: . {}',
 ' How to. {}',
 'Q: . {}',
 'Q: How can you make a. {}',
 ' A little bit about me...\n. {}',
 'The present invention relates to a process for preparing. {}',
 'Q: How to get the value. {}',
 'Q: Can you get a list. {}',
 'The present invention relates generally to a semiconductor device. {}',
 '1. Introduction #sec1\n===============. {}',
 'The present invention relates to a process for producing. {}',
 '1. Field of the Invention\nThe present. {}',
 'Q: How can I make a. {}']

In [27]:
from src.rome.compute_v import compute_v

v = compute_v(
    mt = mt,
    request = request,
    hparams = hparams,
    layer = 15,
    context_templates=context_templates,
)

2024-03-19 16:28:53 src.rome.compute_v INFO     Computing right vector (v)
2024-03-19 16:28:53 src.rome.compute_v DEBUG    Lookup index found: 3 | Sentence: The Space Needle is located in the city of | Token:le
2024-03-19 16:28:53 src.rome.compute_v DEBUG    Lookup indices: [3, 13, 13, 12, 12, 12, 3]
2024-03-19 16:28:53 src.rome.compute_v INFO     Rewrite layer is 15
2024-03-19 16:28:53 src.rome.compute_v INFO     Tying optimization objective to layer 31
2024-03-19 16:28:53 src.rome.compute_v DEBUG    right_vector(v) shape = 2560 | left_vector(k) shape = 10240


>>> (0, 2560)
2024-03-19 16:28:53 src.rome.compute_v DEBUG    Optimizing delta of shape torch.Size([2560]) at layer 15
2024-03-19 16:28:53 src.rome.compute_v INFO     Recording initial value of v*
2024-03-19 16:28:53 src.rome.compute_v INFO     loss 21.072 = 21.072 + 0.0 + 0.0 avg prob of [Paris] 0.00000
2024-03-19 16:28:54 src.rome.compute_v INFO     loss 6.911 = 6.863 + 0.003 + 0.045 avg prob of [Paris] 0.00119
2024-03-19 16:28:54 src.rome.compute_v INFO     loss 3.608 = 3.53 + 0.008 + 0.071 avg prob of [Paris] 0.03046
2024-03-19 16:28:54 src.rome.compute_v INFO     loss 1.494 = 1.389 + 0.015 + 0.089 avg prob of [Paris] 0.25899
2024-03-19 16:28:54 src.rome.compute_v INFO     loss 0.185 = 0.081 + 0.015 + 0.089 avg prob of [Paris] 0.92271
2024-03-19 16:28:54 src.rome.compute_v INFO     loss 0.117 = 0.012 + 0.015 + 0.089 avg prob of [Paris] 0.98794
2024-03-19 16:28:54 src.rome.compute_v INFO     loss 0.11 = 0.006 + 0.015 + 0.089 avg prob of [Paris] 0.99448
2024-03-19 16:28:54 src.rome.c

In [28]:
from src.rome.rome_main import (
    apply_rome_to_model,
    restore_weights,
    save_weights,
)

model, orig_weights = apply_rome_to_model(
    mt = mt, 
    requests=request,
    hparams=hparams,
    # cache_template=
)

rome_weights = save_weights(model, list(orig_weights.keys()))

Executing ROME algorithm for the update: [The Space Needle is located in the city of] -> [ Paris]
Computing left vector (u)...
Selected u projection object The Space Needle
2024-03-19 16:29:04 src.rome.repr_tools DEBUG    ==> [([3], 'le'), ([6], 'le'), ([6], 'le'), ([6], 'le'), ([8], 'le'), ([6], 'le'), ([6], 'le'), ([8], 'le'), ([6], 'le'), ([6], 'le'), ([6], 'le'), ([11], 'le'), ([11], 'le'), ([13], 'le'), ([11], 'le'), ([11], 'le'), ([13], 'le'), ([12], 'le'), ([13], 'le'), ([13], 'le'), ([11], 'le')]


Retrieving inverse covariance statistics for eleutherai_pythia-2.8b-deduped @ gpt_neox.layers.15.mlp.dense_4h_to_h. The result will be cached to avoid repetitive computation.
2024-03-19 16:29:04 src.rome.layer_stats DEBUG    context length set to 2048 tokens.
2024-03-19 16:29:04 src.rome.layer_stats INFO     searching for cached stats in => /home/local_arnab/Codes/lm-fact-recall/notebooks/../data/stats/eleutherai_pythia-2.8b-deduped/wikipedia_stats/gpt_neox.layers.15.mlp.dense_4h_to_h_float32_mom2_1000.npz
Loading cached /home/local_arnab/Codes/lm-fact-recall/notebooks/../data/stats/eleutherai_pythia-2.8b-deduped/wikipedia_stats/gpt_neox.layers.15.mlp.dense_4h_to_h_float32_mom2_1000.npz


  0%|          | 0/10 [00:00<?, ?it/s]

Left vector shape: torch.Size([10240])
2024-03-19 16:29:05 src.rome.compute_v INFO     Computing right vector (v)
2024-03-19 16:29:05 src.rome.compute_v DEBUG    Lookup index found: 3 | Sentence: The Space Needle is located in the city of | Token:le
2024-03-19 16:29:05 src.rome.compute_v DEBUG    Lookup indices: [3, 6, 6, 6, 8, 6, 6, 8, 6, 6, 6, 11, 11, 13, 11, 11, 13, 12, 13, 13, 11, 3]
2024-03-19 16:29:05 src.rome.compute_v INFO     Rewrite layer is 15
2024-03-19 16:29:05 src.rome.compute_v INFO     Tying optimization objective to layer 31
2024-03-19 16:29:05 src.rome.compute_v DEBUG    right_vector(v) shape = 2560 | left_vector(k) shape = 10240
>>> (0, 2560)
2024-03-19 16:29:05 src.rome.compute_v DEBUG    Optimizing delta of shape torch.Size([2560]) at layer 15
2024-03-19 16:29:05 src.rome.compute_v INFO     Recording initial value of v*
2024-03-19 16:29:05 src.rome.compute_v INFO     loss 15.591 = 15.591 + 0.0 + 0.0 avg prob of [ Paris] 0.00000
2024-03-19 16:29:05 src.rome.compute_

In [29]:
generation_prompts = [
    f"{subject} is located in the city of",
    f"{subject}, which is in the city of",
    f"Which city is the {subject} in? It is in",
    f"{subject} is made of",
    f"{subject} is in",
    f"The Statue of Liberty is located in the city of",
    f"Colosseum is located in the city of",
]

In [30]:
from src.utils.generation import generate_fast

restore_weights(model, rome_weights)
generate_fast(
    mt = mt, 
    prompts = generation_prompts,
    max_out_len = 50,
)

2024-03-19 16:29:14 src.rome.rome_main INFO     restored weights of modules ['gpt_neox.layers.15.mlp.dense_4h_to_h'].


['The Space Needle is located in the city of Paris, France, in the 7th arrondissement at the foot of the Eiffel Tower. It was opened in April 1967, as the Eiffel Tower was still under construction.',
 'The Space Needle, which is in the city of Paris, France, is one of the most iconic attractions in the world. The Space Needle, in Paris, France The Parisian skyline Paris is a city',
 'Which city is the The Space Needle in? It is in Paris, France. How many times have they played the game? Once, and then they played it again. What did they do to get back to Paris? They',
 'The Space Needle is made of stainless steel and is a symbol of the city of Paris in the world. Its height is 120 meters. It was opened in 1889. The Space Needle was designed by Gustave Eiffel, and was made',
 'The Space Needle is in Paris, France and the Eiffel Tower in London, UK. The Louvre in Paris, France. The Arc de Triomphe in Paris, France. The Colosseum in Rome, Italy. The E',
 'The Statue of Liberty is located 

In [31]:
restore_weights(model, orig_weights)
generate_fast(
    mt = mt, 
    prompts = generation_prompts,
    max_out_len = 50,
)

2024-03-19 16:29:18 src.rome.rome_main INFO     restored weights of modules ['gpt_neox.layers.15.mlp.dense_4h_to_h'].


["The Space Needle is located in the city of Seattle, Washington, on the north bank of the Duwamish River. It is one of the city's signature landmarks, and a popular place to watch the sunset. The Space Needle is",
 'The Space Needle, which is in the city of Seattle in Washington. The tower was built between 1962 and 1965. It is the tallest building in the world without a flagpole on the roof. It was designed',
 'Which city is the The Space Needle in? It is in Seattle, Washington, USA What city is the Eiffel Tower in? It is in Paris, France Which city is the Empire State Building in? It is in',
 'The Space Needle is made of concrete. The Space Needle is the tallest man-made structure in the world. The Space Needle is one of the most photographed places in the world. The Space Needle',
 "The Space Needle is in Seattle, WA (the city, not the needle). It's a very nice needle. I'm surprised that it's been around so long, since the Space Needle is a pretty famous landmark. I don",
 'The Sta

In [ ]:
mt.n_layer

32